## Sample: Data Encryption using Azure Key Vault / Managed HSM via Spark UDFs

Let's encrypt some data with a Spark UDF and Azure Key Vault! Before we begin, let's make sure the following is set:

* Cluster Configuration:
  * Local Environment Variables for Service Principal
  * Appropriate Library JAR(s) added to the cluster
  
Once completed, let's run some setup. First, we'll load our required PySpark libraries:

In [ ]:
from pyspark.sql.functions import col, udf, expr, lit
from pyspark.sql.types import StringType

Next, let's create a sample dataframe with a few columns:

In [ ]:
data = [{"Category": 'A', "ID": 1, "Value": 121.44, "Truth": True},
        {"Category": 'B', "ID": 2, "Value": 300.01, "Truth": False},
        {"Category": 'C', "ID": 3, "Value": 10.99, "Truth": None},
        {"Category": 'E', "ID": 4, "Value": 33.87, "Truth": True}
        ]
df = spark.createDataFrame(data)
df.createOrReplaceTempView("test")

Then, let's set a literal for the KeyURI: The absolute URI of the Key we want to use for encrypting our data:

In [ ]:
keyURI = "https://dfkyevault01.vault.azure.net/keys/columnKey/6e9e41d250e045ebbcca420532b2692b"

Then let's register the UDFs. We need the name of the class we want to use (in this case, one class for encrypting, one for encrypting), and the namespace of the package

In [ ]:
spark.udf.registerJavaFunction("encrypt", "com.microsoft.solutions.keyvaultcrypto.doEncryption", StringType())
spark.udf.registerJavaFunction("decrypt", "com.microsoft.solutions.keyvaultcrypto.doDecryption", StringType())

### Encrypting Data

Let's take our sample dataframe from above, and encrypt the category column. We'll use ```expr()``` to pass in our column name and the key URI value:

In [ ]:
encrypted_df = df.withColumn("EncryptedCategory", expr("encrypt(category,'{}')".format(keyURI)))

In [ ]:
display(encrypted_df)

Category,ID,Truth,Value,EncryptedCategory
A,1,true,121.44,TcNk4c8mAr4vhpyExLdc+6jQrRzDTQ8g0zhmgx351fvd2P3Jvf3d9j9Z+M8R7eZAezcoGfJQegqMTVxfOEB8tpup+1OsjS3HIeq4pyrrbD9Q+MRG4h5wg5riEpkMw/DdwKWukWuHW/VMiw/nfc0rA5+ycJrAoWer425AV4+bUDNYSeOjRlOYXQiNy/QrRAj9NKDVitNBsXcKLCvUt+ZrRg+jQ/lVK/eUgbGwzP9TGl7MD/uLymkQH5z2aaeP8rVn1gxSFhSf6599DqwTtvK5fz1EwEohVPOYt6Rmlrn29nwvVahkPFEz24h4DzrXud4dLdKhrhQDzUANQ8WIXUOL8Q==
B,2,false,300.01,gScuqTW4wJ5RRPNjNUGtfjh8hnqPWgA4vqX0iLe/GW2u/pYCVXc3gBlHm+xDT1EnWlEZZiujidjg+k2v9QcxEXLTAOb92YlI3jxoIthV4YZIjMljWi1Etv8B5HF7z0sVg6I//2IDelY7XRTqDFKhpLYeuvwM+FPArtQvWffYvF/4BemfKfJt+TkzUwXStgyEsHhf08uWvDh5uEUhwALoD0echX12uafDVFZ/ArU3FeH2AEpsxnTaPjzmTfbbKq1gxcp2vwnTTTNH7QUtYKVVTrMQ+3Gbq6JxnsON1wD8ft6MTwPJlvSdodcWspBfSkSRzkPcV08UH79CUnGcqhCU6A==
C,3,null,10.99,CrM03XZyDT3bGbsJ891oBips+XFWwiWn+1U5/AAKAS5km92AQj0l532bbHZr7fGPWX5575F+9vj+vF3O466QyanxtLzAn55Qgatq1JYzkE1pb+4o+4tBbl6pkvk50k0JPFIV9d4BWD9ejHqBaMUAjah2Llu2ERSAHBA9ArkZaT//dwro7EvHTKP8NUoGozLEjm0DzhzggPpTnkpSV5UfqFPjn/O/hPst/FEcdWUhWXacYBZkuvv8UKSFR9iAPRAsn+WPBCh7/yLsWkDK0sDsPVFuQ1y4SN3lG5jm1YYNLqhMVQuvDKvm5jnsRM3TV/+qbLMpce5q34oHB80gDfhpgw==
E,4,true,33.87,jcOVEtzfd++CS7bghuB9NuYW9u8gyaLj4uduJm3MbipLScHtP9Purv5S8ieOCLet+vrRFh0VOmxLVGarq31vnxMNki/gZ+iSm5sGUmkr5hLYNznNTWmxiG/omiYstkOkdlpgaz85IZqslG/xR+XjPIE+FMZhejz3heGEQvi3CPsWDpbxq7v+cOxXulD2pGz4S/3o0vAxW4+H2we5MEeYs5GJH3kGlmct6E//KQzzst0rrOV6oueJzvCXkCWJrKnS2/og3g+s6o2wPnRJP8khv2mznQx9v8fzGqyjwpdx6StDhOmsEY3QPpTezWnww0TuN+ms/y92lJtMc0F4Tg2m0A==


### Encrypting With Spark SQL

You can also use these functions in the Spark SQL API or with the magic %sql commands/SQL cells. However, since our function expects a key value we need to set a variable in our Spark configuration, then use that with substituion in your SQL query. Other methods do exist, of course.

In [ ]:
spark.conf.set("crypto.keyURI", keyURI)  

In [ ]:
%sql

SELECT *, encrypt(Category, '${crypto.keyURI}') as EncryptedCategory 
FROM test

Category,ID,Truth,Value,EncryptedCategory
A,1,true,121.44,Gkc3f4xiRk7MAdByIQonJFj5YHmJgyOG9JD2Jgrq/K3CITZaKmt4oFvqp3UMvYSwfUG0pW1E2ah0oMp0RVbAogQUz3bR6kMRZ4kCyIGVGHxFnFfvG2S26JECzfiQkwcHR89Llph7O3xy6Uxcz0lrJHbBZDZIyHwvIKEqmA6EAoJQNEmsdxRMMpOYrCJArwB7kMIlld1XyBDmKmagk5Nl8HnK0LlbSWh0MEHu5ltpryIyPWk2awFZnWxbXxXI0W4jjbrHAsp+Ge9ZY8rhE+JxUxMlPGOOfyVVzroRLKYoCOhztX7iuXAbR3pZqAMSSkc9XXR2K9z7GJvf/L/DEjIcXA==
B,2,false,300.01,Az8vBxYvbqsWKctvMO1RE3eSnB6qoGYrTV7uiqd7e1AXvSFdMLZzoOuK5xEMYLSDgY3+5g64Sjsp/noO9wlulV+ZI66SYngQ9S2T4/dk7i2ZsJ2ctBDo4gwaiGRi7EbhxhT8qgjEN0wTgPF6lXOLHYNJFf19YeNWISekHC8fxd9JDIauCI8T8u6plG+2Pl0sDihP1as4la/e+Kqj25ACDoUlSSO/L87IBkB0h8Lbjr/tQOEpJJtXVG+7IFVDGkBeNxD1m2a/9QnAQhqj+pA4sozJ5XPuJbaI/QzReKmSlf9L9TK09GdBuiZUdEyPD8X1PChsRsOz4BKzfqbPiGBgAg==
C,3,null,10.99,GNG0hFNPbJVUFS9X5ukgsDU+A94wokj8JK05dqCOEDj0/gg9rjtLF/iZFM6SWkHnorJo8PeN6ateX0Ter6ObL33g2ry2SaSLLTR0NE41zqyIRBwBD70T8YiX/BENdjZWMaG1xm4Te3WbGf7jDQBctcn08P2F1LlzRAXY6GJ+XJNcaNnmlxtdpfDUGQoAtPDeaHznZXMzn6NYqcI16Wv5uA/TsfxUqxHBp+6nXqs4sQcALsD8zzjUmDYVP4NdfnnDZ97HVD4AyJd3sGPMU4GY+YWPg1+4n4up3xzI4JO5QvbkmVjg0FzlduVp9NrUr7BqYJ0FqLF1xuByYeo8IP+IkQ==
E,4,true,33.87,bskt3GlFNJbJuBvfXRAizs07NsWEcUx6ueUDwTpZuY3RN0ktcFUucbmPi+Wd+f03ZFT2Kdz0aJ+F6c2/bZ6CbIzrA5NCg+gfob6sV7wJU63i0gfgDRjhb9gKgu+r9HNJ0ILXQX17JDTVggV6ptAKO5LlbVZEfNeDMJ0tue2LB3XMomsU+o0sA+b6CZr85SrbSRfKSLwtnSa2USbuALCtVeO8CvoXGS8rYTU6nahcLcqx43/mnVMHdnj2KIPET556p5B87/obMwEkxL+9ajrSD1Z6kdx+KhJqOC2zgM9n2zUNV8TvCG4jiN4CGX/5JJKn0wF/ddOeeVFlmrlX4D/3IQ==


## Decrypting Data

This works much the same way, just using the other function and passing in the encrypted value.

In [ ]:
decrypted_df = encrypted_df.withColumn("DecryptedCategory", expr("decrypt(EncryptedCategory,'{}')".format(keyURI)))

In [ ]:
display(decrypted_df)

Category,ID,Truth,Value,EncryptedCategory,DecryptedCategory
A,1,true,121.44,JszzlKRngJKUcosMGUrw0GkfM0J47lxFhU8lsOYVmKiBi6WFxDRzjtMwDLeQPKwyZiQ/j+v9BgbcqVgPTNA9PrRMgDWe1YjbjgChMJrJ/OTzBh3Ipdi/rZgu8HWiC9BuZQVut1xEpFb/1yHKqF17xHJ0jynrzjXSkNXl2G+SprdoAy/Se4DXcvveduXJGnZGn2ChH210xz8iJ7XT3j55yyGqlVHORHsn8BgaXwVBSkZxVf/0Hz8fhTL9M33oM7TlvV8fOArPmuRlJVzRdiMUqpE9GjKPWrKr4HLinV+CJBIOqdzc0e8eiELoCfDBpJnFO1jtYhN9jv/ESz00uTxotw==,A
B,2,false,300.01,OZ7XjCV71XOWu4R0bBiE3uQdUFxu28BUXqjJvQFRefKu3T2doHHotUZ6mqnE/R+nHgVMz/glrxHLyAL9pSWtPV2uiu/liTknCdOkIyeO4NVXoO9pbEy+r+QPCjz1zl/zIPM6XYdBjl+CV8GwwvfbshwT7+wh7s0HkhUECzxp74w2YUzQtWW2lpnJZCiOO1EUH0D/NF0FCc6DDFU5BFxRXjU0CPXE4w48O/xwMb7q5EWv3TV7SdRjwHVljmJUxFukrpkTd+aeP9UBPNnGpqNHIPy35DLgs4ziPdsY3ie73+nuOH4kJ4uLnP4WGLZ9KZwaLsJic5UVT4PlDIYZh3NBkQ==,B
C,3,null,10.99,DyqcifvGbe9KR1egaoJWPHzUhQ6FkQli2ECvjQ2tJCBSCHn240VBfUrKxAYO7kbPxlGKcSO/JH32D1yvGt9JlnwxykC8gcG4RMNpNFeqU4kPBLdKYUNP00ZJR/vcX/Y7xjE/v7sJm/6PrVGp5i2lXHJU1MMGNMC78VuRn/8iWbfl5zs9xBwF0+xLVQi+cy40xyJkYcShSH1GfG38ajiV8TPHRy3r77Ql9jA7fBP8X0xhGT/xF14wuQ86PWZmOyql5jNdv5utYIJvatMU5/0JOp2ZXvJ7szFtXdgWnjKHNGfKCu/JRqSJFe9fmHIaF8cqrhNdMjrgfo12BOrRv/Q1eQ==,C
E,4,true,33.87,aO2Hc1Snz1kiq0WpMljdXffAmVViM6JCVfYMX3MIUVduZ0TVfHcozyZqvnwfvuWKVt2sFwWA5vQ93ZQx058COH+R6YehX16kBsb00qhc9INQnVKExC5tmd2c5QGHgJAVAoAmCilNubal6cWvMMr1FGC2xDkTnhU4Z4A6xPg+F2sAKaYjyAxI+dW2wSrIxCwllW0sPgYhtdDv7ADtPFikZA3jFaKRxGLSn3YfPoVg+aCAe6Obf5Egd9i0IZwrow22j/9VusT1sTYi2pX6M3wMq4KhxUSZl/RmHs10AnnnbDN82sSQ1ZTP3tG+8/xldAFOGT0uY7pJPw4N8khbT+Si3w==,E


### Decrypting With Spark SQL

The same method can be used to decrypt with Spark SQL. In this example we'll refresh our 'test' view with the encrypted dataframe, then run the query

In [ ]:
encrypted_df.createOrReplaceTempView("test")

In [ ]:
%sql

SELECT *, decrypt(EncryptedCategory, '${crypto.keyURI}') as DecryptedCategory 
FROM test

Category,ID,Truth,Value,EncryptedCategory,DecryptedCategory
A,1,true,121.44,QOXBp/yUp0STt/Wefo8JGibNWstFLHZ3K3H60BV23SGsR6TzRKV0TJ9sL4WkYQ2T9w1+Y8eGIB/ze0Ud3otnek65GDyMP2iZ4GXRzY1e+m70qefKfIi7kfBm/j8uFY1NElyMF8hlU5GO0QXokyf2irQmw8eH0L9HyxP4wVOE4SmR8gefwivtwn/11F1z/Wzaq8f1b7WkLlD8so+tg6BNzmYbIwT7zmntPK44q/+rgoXF1BX+TglxCGqz2sADGqaiSnoy/hWO0qcfIF0x50Q0RU4b1x3quSR/stK1b5rkEPndaaER+g2g/c+wgQMI5Xgi75w/sXF6rV14ERHHwKehtQ==,A
B,2,false,300.01,gy4tjWVSll4fDAI1HtzFov9FfhweM9eQ9ghK+vcXAlBF5NObzPTNoC0PlZjf9sUGgJHTBohckWp/gwFIVAHizGmZOJO6ONvoMUt54fsh9EG0zJtKSER9ndyis3rnd4sPXV17X+P5GqHV0c6ZCXFXB5NxaxIuUPXyUfnyjvs+CDCUCxoaS4EDj3SryrKI5pjeGK4ZT12Mfs6xezf5VHv4K0d/wCYSzUgc1b4Nb3VId212ejMCOg8N/cYMKr/wgLBbGqVgRkUg4xzyM6lRmcGduip02UvkyqYNrMvaQR7zyxukqXjxfT7OSBL+9KkUVqoBfUiKOXJIC5f2ytjmP0JiVA==,B
C,3,null,10.99,jKPWapCeBgOj+jcb1d6v5W2S7kUDBt/LKP0quCrF0vphz1kt2nh9jZdgmFuHIyKrsvgHXkUAgEeGAxynQ1V1XaSiTmrWP19uADXu1a/ZXHsRYSoG+AvYx1xeAEFudHdgTdgDWUcYaf8XCDfEe9UDbVUXDlK2nJyqkiF479ADmdmfLTYrcMK0O31S9RXtrVHh5kSUfThuPp1VNDeZuf7ULqJQmRlgQh5ToscsC64m0ZcGMIQsnCdmHx7j6nl98MEB0f6y7Qb/BOtFoxVBc79r4PjS87R3a0omR59NUM91SETc/ygGlEYI/oy3zVDXtjEQQY0DRJ6VPi1xNd/jOeTEDg==,C
E,4,true,33.87,VYIim++GUMeREsEyc3uaNrPMazL7uNHc0hnYqoCd4dVUOFx/ASBrmrxqHR+n2MgwdmV9SGTBD2m3jQXv2nXZH4deaVxylvA/yOuVIS6gvL8wp/uVc3iQ0/JcYm1vG0uWXfueIDE6CwAwM6kD5uGHhFcXB8KyMp93++KpO5VHSqaRkev8Zsd625D4GflH9KrF7hYbfkQBzD8l9flvvIiIZ+pU4AogVH7hjjIGPbMOqs8TZ/sKwKTlRaPOSoLU5v8Vw3fgcngOkY7qxrzJoHe7XHKK6SlxJeCZQyVczoM044VbkyWgzLE2UsrRBkbTK/VF4k6+YGLuawQB0WoQuM1DAg==,E
